## Object Detection and Classification Using TensorFlow (CPU) on Windows 10.

### Spring position detection and classification

This notebook explain how I have trained an Object Detection and Classification API using my own data. In my case I will be detecting and classifying springs position of a valve used in the assembly of gas cook-tops.

This jupyter nootebook is based completely in a tutorial called "How To Train an Object Detection Classifier for Multiple Objects Using TensorFlow (GPU) on Windows 10" done by Evan Juras. His tutorial is great and well detailed. This notebook is a simplified version of his guide applied to a real problem placed in a manufaturing line. For more details about his didactic document follow the link below:

https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10



In a future I am planning to translate his tutorial to Spanish and Portuguese as I think his work is great and must be shared.

<img src="Images_API/BOTH_29.png" alt="Drawing" style="width: 250px;"/>

#### Downloading the necessary files to build use the object detection API.

At this point you must go to the tutorial of Evan Juras at https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10 and follow his steps in how to download and organize all the files necessary to build the object detection API.

P.S. From this point I haven't followed exactly his steps because I have changed some details of his tutorial. These details are referent to the python virtual env and folder names and I also changed his file "Object_detection_image.py" to adapt to my problem and to the outputs I want.

#### Create a Python Virtual Environment

In the anaconda prompt I typed the follow command. It creates an virtual environment called "ComputerVision" in Python.

```console
C:\> conda create -n ComputerVision pip python=3.7 
```

#### Activating the Python Virtual Environment and update pipe

```console
C:\> activate ComputerVision

(ComputerVision) C:\>python -m pip install --upgrade pip
```

#### Installing tensorflow CPU

```console

(ComputerVision) C:\> pip install --ignore-installed --upgrade tensorflow
```

#### Installing the necessary packages.

I had problems installing protobuf. The only version that worked for me was the 3.6.

```console

(ComputerVision) C:\> conda install -c anaconda protobuf=3.6
(ComputerVision) C:\> pip install pillow
(ComputerVision) C:\> pip install lxml
(ComputerVision) C:\> pip install Cython
(ComputerVision) C:\> pip install contextlib2
(ComputerVision) C:\> pip install jupyter
(ComputerVision) C:\> pip install matplotlib
(ComputerVision) C:\> pip install pandas
(ComputerVision) C:\> pip install opencv-python

````

#### Configure PYTHONPATH environment variable

```console
(ComputerVision) C:\> set PYTHONPATH=C:\ComputerVision\models;C:\ComputerVision\models\research;C:\ComputerVision\models\research\slim
```

#### Compile Protobufs and run setup.py

From the folder 
```console
(ComputerVision) C:\ComputerVision\models\research
```
I have done:
```console
protoc --python_out=. .\object_detection\protos\anchor_generator.proto .\object_detection\protos\argmax_matcher.proto .\object_detection\protos\bipartite_matcher.proto .\object_detection\protos\box_coder.proto .\object_detection\protos\box_predictor.proto .\object_detection\protos\eval.proto .\object_detection\protos\faster_rcnn.proto .\object_detection\protos\faster_rcnn_box_coder.proto .\object_detection\protos\grid_anchor_generator.proto .\object_detection\protos\hyperparams.proto .\object_detection\protos\image_resizer.proto .\object_detection\protos\input_reader.proto .\object_detection\protos\losses.proto .\object_detection\protos\matcher.proto .\object_detection\protos\mean_stddev_box_coder.proto .\object_detection\protos\model.proto .\object_detection\protos\optimizer.proto .\object_detection\protos\pipeline.proto .\object_detection\protos\post_processing.proto .\object_detection\protos\preprocessor.proto .\object_detection\protos\region_similarity_calculator.proto .\object_detection\protos\square_box_coder.proto .\object_detection\protos\ssd.proto .\object_detection\protos\ssd_anchor_generator.proto .\object_detection\protos\string_int_label_map.proto .\object_detection\protos\train.proto .\object_detection\protos\keypoint_box_coder.proto .\object_detection\protos\multiscale_anchor_generator.proto .\object_detection\protos\graph_rewriter.proto .\object_detection\protos\calibration.proto .\object_detection\protos\flexible_grid_anchor_generator.proto
```

then:

```console
(ComputerVision) C:\ComputerVision\models\research> python setup.py build
(ComputerVision) C:\ComputerVision\models\research> python setup.py install
```

#### Test TensorFlow setup to verify it works

At this point you must go to the tutorial of Evan Juras at https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10 and follow his steps to test if the API is working correctly.

#### Gather and Label Pictures

After taking pictures of valves with springs in the right position and with inclined springs it was time to label these pictures and for that I have used "LabelImag". More information about it you can find at https://github.com/tzutalin/labelImg#labelimg.

After labelling the images I have one xml file for image corresponding to where the object I want to detect and classify was and its label.

<img src="Images_API/labelimg.png" alt="Drawing" style="width: 1000px;"/>

After generating the xml file for each image I put them into:
```console
C:\ComputerVision\models\research\object_detection\images
```
In the corresponding folder, train or test. I have used 20% of my images for test.

#### 4. Generate Training Data

After putting the images into the right folders I run:

```console
(ComputerVision) C:\ComputerVision\models\research\object_detection> python xml_to_csv.py
```
It generates 2 csv files into the folder "images", one corresponding for test and other for train.

#### Changing some files

I have used notepad++ to make some changes in some files to adapt them to my problem. The first files to be changed is "generate_tfrecord.py" where I replaced the label map existing to my map (line 31). As I wanna identy if the spring is moved or not I am predicting two classes, OK and NOK.

<img src="Images_API/labelmap.png" alt="Drawing" style="width: 500px;"/>

#### Generate the TFRecord files

```console
(ComputerVision) C:\ComputerVision\models\research\object_detection> python generate_tfrecord.py --csv_input=images\train_labels.csv --image_dir=images\train --output_path=train.record
```

```console
(ComputerVision) C:\ComputerVision\models\research\object_detection> python generate_tfrecord.py --csv_input=images\test_labels.csv --image_dir=images\test --output_path=test.record
```

#### Create Label Map and Configure Training

Using notepad++ I have created a file called labelmap.pbtxt and I put it into the folder
```console
C:\ComputerVision\models\research\object_detection\training
```

This file contains my classes map. The sctructure of this file can be seen below.

<img src="Images_API/labelmappb.png" alt="Drawing" style="width: 1000px;"/>

#### Configure training

I have copied the file "faster_rcnn_inception_v2_pets.config" from the folder
```console
C:\ComputerVision\models\research\object_detection\samples\configs
```

And I past it into
```console
C:\ComputerVision\models\research\object_detection\training
```

Then I open the file "faster_rcnn_inception_v2_pets.config" that is into the training dir using notepad++ and have done few changes on it.

<li>Line 9. I have changed num_classes to the number of different objects I have, in this case 2.</li>

<li>Line 106. I have changed the fine_tune_checkpoint:

```console
fine_tune_checkpoint: "C:/ComputerVision/models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt"
```

<li>Lines 123 and 125. I have changed the input_path and the label_map_path:

```console
input_path: "C:/ComputerVision/models/research/object_detection/train.record"
```

```console
label_map_path: "C:/ComputerVision/models/research/object_detection/training/labelmap.pbtxt"
```

<li>Lines 130. I have changed the num_examples to the number of images I have in the \images\test directory, in this case, 185.

<li>Lines 135 and 137. I have changed the input_path and the label_map_path:

```console
input_path: "C:/ComputerVision/models/research/object_detection/test.record"
```

```console
label_map_path: "C:/ComputerVision/models/research/object_detection/training/labelmap.pbtxt"
```

#### Run the Training

From the object_detection folder I have run:

```console
python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_pets.config
```

To be able to check how the model evaluation would be over time I have used tensorboard. To be able to do this I opened a new anaconda prompt without closing the other that was running the model and run:

```console
(ComputerVision) C:\ComputerVision\models\research\object_detection>tensorboard --logdir=training
```

#### Results

After running more than 10k epochs I got a loss of around 0.1. And it is a great result.

<img src="Images_API/Loss_Inception.png" alt="Drawing" style="width: 500px;"/>

After training the RCNN I have done some tests and it have showed great results, thus I started to use this model in the production line. In 5 days it was produced around 35k products and the model has detected 3 products presenting springs out of position and in all the cases it detected correctly.

<img src="Images_API/Detection_4.png" alt="Drawing" style="width: 700px;"/>